In [ ]:
%load_ext autoreload
%autoreload 2  

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')) # or the path to your source code
sys.path.insert(0, module_path)

In [ ]:
def calculate_review_sentiment_score(x):
    tot = 0
    card = 0
    for word, _, score in x:
        if score != 0:
            tot += score
            card += 1
    
    return tot/card if card != 0 else 0    

In [ ]:
from ast import literal_eval
dataset['tagged_text'] = dataset['tagged_text'].apply(literal_eval)

I ratings sono da 1 a 5, mentre i sentiment scores da -5 a 5, quindi vengono normalizzati tra 1 e 5 

In [ ]:
def normalize_RSS(x):
    norm = 0
    if x >= 3:
        norm = 5

    elif 1 < x <= 3:
        norm = 4

    elif -0.5 < x <= 1:
        norm = 3

    elif -3 < x <= -0.5:
        norm = 2

    elif x <= -3 :
        norm = 1

    return norm

In [ ]:
if 'RSS' not in dataset.columns:
    dataset['RSS'] = dataset['tagged_text'].apply(calculate_review_sentiment_score)
    dataset['RSS'] = dataset['RSS'].apply(normalize_RSS)

In [ ]:
dataset

In [ ]:
dataset['RSS'].value_counts()

In [ ]:
import plotly.graph_objects as go

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=dataset['RSS'].value_counts().index, values=dataset['RSS'].value_counts().values, hole=.3)])
fig.show()

BERT per migliorare la fase di SA. Nel paper originale viene utilizzato il lexicon AFINN, che essendo appunto un aprroccio lexicon based ha molte limitazioni rispetto a BERT.

In [ ]:
import pandas as pd
import torch

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

link alla descrizione del modello utilizzato: https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment#bert-base-multilingual-uncased-sentiment

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment").to(device)

In [ ]:
def get_input_ids_and_attention_mask_chunk(tokens):
    """
    This function splits the input_ids and attention_mask into chunks of size 'chunksize'. 
    It also adds special tokens (101 for [CLS] and 102 for [SEP]) at the start and end of each chunk.
    If the length of a chunk is less than 'chunksize', it pads the chunk with zeros at the end.
    
    Returns:
        input_id_chunks (List[torch.Tensor]): List of chunked input_ids.
        attention_mask_chunks (List[torch.Tensor]): List of chunked attention_masks.
    """
    chunksize = 512
    input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
    attention_mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))
    
    for i in range(len(input_id_chunks)):
        input_id_chunks[i] = torch.cat([
            torch.tensor([101], device=device), input_id_chunks[i], torch.tensor([102], device=device)
        ])
        
        attention_mask_chunks[i] = torch.cat([
            torch.tensor([1], device=device), attention_mask_chunks[i], torch.tensor([1],device=device)
        ])
        
        pad_length = chunksize - input_id_chunks[i].shape[0]
        
        if pad_length > 0:
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.tensor([0] * pad_length, device=device)
            ])
            attention_mask_chunks[i] = torch.cat([
                attention_mask_chunks[i], torch.tensor([0] * pad_length, device=device)
            ])
            
    return input_id_chunks, attention_mask_chunks 

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode_plus(review, add_special_tokens=False, return_tensors='pt').to(device)
    input_id_chunks, attention_mask_chunks = get_input_ids_and_attention_mask_chunk(tokens)
    input_ids = torch.stack(input_id_chunks)
    attention_mask = torch.stack(attention_mask_chunks)
    input_dict = {
        'input_ids' : input_ids.long(),
        'attention_mask' : attention_mask.int()
    }
    
    outputs = model(**input_dict)
    probabilities = torch.nn.functional.softmax(outputs[0], dim = -1)
    mean_probabilities = probabilities.mean(dim = 0)
    
    return torch.argmax(mean_probabilities).item() + 1

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
%%time
if 'BERT_RSS' not in dataset.columns:
    dataset['BERT_RSS'] = dataset['reviewText'].progress_apply(sentiment_score)

In [ ]:
import plotly.graph_objects as go

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=dataset['BERT_RSS'].value_counts().index, values=dataset['BERT_RSS'].value_counts().values, hole=.3)])
fig.show()

In [ ]:
#dataset.to_csv('data/preprocessed_dataset.csv', index=False, sep=';')

In [ ]:
# Imposta la larghezza massima per la visualizzazione delle colonne
pd.set_option('display.max_colwidth', None)

In [ ]:
dataset[(dataset['BERT_RSS'] == 1) &( dataset['RSS'] >= 3)][['BERT_RSS','RSS','preprocessed_text']].head(5)

NLTK Punkt Sentence Tokenizer per splittare le reviews in sentences

In [ ]:
import nltk
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') #sentence_tokenizer

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
from afinn import Afinn

afinn = Afinn()

In [ ]:
# Funzione per splittare le recensioni in frasi
def preprocess_analyze_sentences(review, sa):
    sentences_sentiments = {}
    sentences = sent_tokenizer.tokenize(review)
    
    for sentence in sentences:
        afinn_scores = []
        sentence_clean = re.sub(r'[^\w\s]', '', sentence) #cleaning
        # Tokenizza in parole
        words = nltk.word_tokenize(sentence_clean)
        # Rimuovi stopwords e applica lemmatization
        preprocessed_words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in stop_words]
        for word in preprocessed_words:
            score = afinn.score(word)
            if score != 0:
                afinn_scores.append(score)
        # Ricostruisci la frase preprocessata come stringa
        preprocessed_sentence = ' '.join(preprocessed_words)
        # Calcola il sentiment
        if(sa == 'afinn'):
            sentences_sentiments[preprocessed_sentence] = normalize_RSS((sum(afinn_scores)/len(afinn_scores))) if len(afinn_scores) != 0 else normalize_RSS(0)
        elif(sa == 'bert'):
            sentences_sentiments[preprocessed_sentence] = sentiment_score(sentence)
        
        #sentences_sentiments.append((preprocessed_sentence, (sum(afinn_scores)/len(afinn_scores)) if len(afinn_scores) != 0 else 0))

    return sentences_sentiments


In [ ]:
# Funzione per combinare tutti i dizionari di sentences per ogni 'asin'
def combine_sentences(sent_dicts):
    combined_dict = {}
    for d in sent_dicts:
        combined_dict.update(d)
    return combined_dict

In [ ]:
r = """Motorola Q Sucks. If you are a serious user of cell phone, then don't buy this.
The phone is not good. That is the reason, Verizon has brought down the price from """

preprocess_analyze_sentences(r, sa='bert')

In [ ]:
# Aggiungere il campo features basato sulla lista di features
def extract_features(sent_dict, features_list):
    feature_sentiments = {}
    for feature in features_list:
        feature_values = [sentiment for sentence, sentiment in sent_dict.items() if feature in sentence.split()]
        if feature_values:
            feature_sentiments[feature] = sum(feature_values) / len(feature_values)
    return feature_sentiments


In [ ]:
# %%time
filtered_features = pd.read_csv('data/ontology_2_filtered.csv', sep=";")
# Applicare la funzione a ogni recensione e raggruppare per 'asin'
dataset['preprocessed_sentences'] = dataset['reviewText'].progress_apply(preprocess_analyze_sentences, args=('bert',))

# Raggruppare per 'asin' e combinare le sentences
grouped = dataset.groupby('asin')['preprocessed_sentences'].progress_apply(list).reset_index()
grouped['preprocessed_sentences'] = grouped['preprocessed_sentences'].progress_apply(combine_sentences)
grouped['features'] = grouped['preprocessed_sentences'].progress_apply(lambda x: extract_features(x, filtered_features['value']))

result = grouped.to_dict(orient='records')

In [ ]:
#Salva il risultato in un file JSON
with open('data/bert_features_scores2.json', 'w') as f:
      json.dump(result, f, indent=4)

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import json
import seaborn as sns

In [ ]:
from nlp_project.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR, FIGURES_DIR
from nlp_project.sentiment_analysis import json_sentences_and_features_scores

# BRAND ANALYSIS

In [ ]:
dataset = pd.read_csv(PROCESSED_DATA_DIR / 'preprocessed_dataset.csv', sep=';')
dataset

In [ ]:
dataset_top5_brands = dataset[dataset['brand'].isin(['Samsung', 'Motorola', 'Apple', 'BlackBerry', 'LG'])]
dataset_top5_brands

In [ ]:
import plotly.express as px

# Raggruppiamo le recensioni per anno e contiamo quante ce ne sono per ogni anno
reviews_per_year = dataset_top5_brands.groupby('year').size().reset_index(name='count')
# Creiamo il grafico a barre con Plotly
fig = px.bar(reviews_per_year, x='year', y='count', title='Numero di Recensioni per Anno', 
             labels={'year':'Anno', 'count':'Numero di Recensioni'}, 
             text='count')

# Mostriamo il grafico
fig.show()

In [ ]:
df = dataset_top5_brands[dataset_top5_brands['year'] >= 2013]

In [ ]:
# Funzione per contare le occorrenze delle features in una review
def count_features_in_review(review, features):
    feature_count = {}
    for feature in features:
        # Trova il numero di occorrenze della feature nel testo della review (case insensitive)
        feature_count[feature] = len(re.findall(r'\b' + re.escape(feature) + r'\b', review.lower()))
    return feature_count

In [ ]:
features = pd.read_csv(PROCESSED_DATA_DIR / 'ontology_filtered.csv', sep=';').iloc[:,0].tolist()

In [ ]:
# Applica la funzione alle review per contare le features
df['feature_counts'] = df['reviewText'].apply(lambda x: count_features_in_review(x, features))

# Trasforma il dizionario di conteggi in colonne individuali per ogni feature
feature_df = pd.DataFrame(df['feature_counts'].tolist(), index=df.index)

# Aggiungi le colonne delle features al dataframe originale
df = pd.concat([df, feature_df], axis=1)

#Raggeùruppa per brand e sommare le occorrenze delle features
brand_feature_freq = df.groupby('brand')[features].sum()

In [ ]:
brand_feature_freq

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

top_features = {}
k = 5
# Itera su ciascun brand e crea un plot con le 5 features più frequenti
for brand in brand_feature_freq.index:
    # Ordina le features per frequenza e prendi le 5 più frequenti
    top_10_features = brand_feature_freq.loc[brand].nlargest(k)
    top_features[brand] = top_10_features.index.to_list()
    # Creazione del grafico
    plt.figure(figsize=(8, 5))
    sns.barplot(x=top_10_features.values, y=top_10_features.index, palette="Blues_d")
    
    # Aggiungi titolo e label
    plt.title(f'Top {k} Features for {brand}', fontsize=16)
    plt.xlabel('Frequency', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    
    # Mostra il grafico
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / f'{brand}_most_cited_features.png', format='png')  # Salva con il nome del brand

    plt.show()


In [ ]:
top_features

## come è cambiato nel tempo il sentiment nei confronti di queste features, per ciascun brand?

In [ ]:
# Dizionario per salvare i DataFrame per ciascun brand e anno
brand_year_dfs = {}

for brand in df['brand'].unique():
    for year in df['year'].unique():
        # Filtra il dataframe per il brand e l'anno corrente
        filtered_df = df[(df['brand'] == brand) & (df['year'] == year)]
        
        # Salva il dataframe in un dizionario con chiave come (brand, year)
        brand_year_dfs[(brand, year)] = filtered_df

        # Opzionale: puoi stampare la dimensione del dataframe o una preview
        #print(f"DataFrame per {brand} nel {year}:")
        #print(filtered_df.head())

In [ ]:
#brand_year_dfs[('Apple', 2018)]

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from afinn import Afinn

afinn = Afinn()
lemmatizer = WordNetLemmatizer()
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') #sentence_tokenizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')


stop_words = stopwords.words('english')

In [ ]:
#%%time

def calculate_features_scores_brand_year(brand_year_dfs, filtered_features): 
    for (brand, year), df in brand_year_dfs.items(): 
        print(f"Calcolo delle features per {brand} nel {year}")
        json_sentences_and_features_scores(df,filtered_feautes, lemmatizer, stop_words, afinn, sent_tokenizer, save_path=f'brand_analysis/bert_features_scores_{brand}_{year}.json')
        # Applicare la funzione a ogni recensione e raggruppare per 'asin'
        #df['preprocessed_sentences'] = df['reviewText'].progress_apply(preprocess_analyze_sentences, args=('bert',))
        
        # Raggruppare per 'asin' e combinare le sentences
        #grouped = df.groupby('asin')['preprocessed_sentences'].progress_apply(list).reset_index()
        #grouped['preprocessed_sentences'] = grouped['preprocessed_sentences'].progress_apply(combine_sentences)
        #grouped['features'] = grouped['preprocessed_sentences'].progress_apply(lambda x: extract_features(x, filtered_features['value']))
        
        #result = grouped.to_dict(orient='records')
        #Salva il risultato in un file JSON
        #with open(f'data/bert_features_scores_{brand}_{year}.json', 'w') as f:
        #  json.dump(result, f, indent=4)

In [ ]:
#calculate_features_scores_brand_year(brand_year_dfs, features)

In [ ]:
import json
import os
import pandas as pd
from collections import defaultdict

# Definisci la directory che contiene i file JSON
json_directory = PROCESSED_DATA_DIR / 'brands_analysis/'

# Dizionario per salvare le medie delle features per ciascun brand e anno
brand_year_averages = {}

# Itera su ciascun file JSON nella directory
for filename in os.listdir(json_directory):
    if filename.endswith(".json"):
        # Estrai brand e anno dal nome del file (es: Apple_2018.json)
        brand, year = filename.replace(".json", "").split('_')
        
        # Carica il file JSON
        with open(os.path.join(json_directory, filename), 'r') as f:
            data = json.load(f)
        
        # Dizionario per accumulare i punteggi delle features
        feature_scores = defaultdict(list)
        
        # Itera su ciascun prodotto nel file JSON
        for product in data:
            # Prendi il dizionario delle features e i relativi punteggi
            features = product.get('features', {})
            
            # Accumula i punteggi per ciascuna feature
            for feature, score in features.items():
                feature_scores[feature].append(score)
        
        # Ora possiamo calcolare la media per ciascuna feature
        feature_means = {feature: sum(scores) / len(scores) for feature, scores in feature_scores.items()}
        
        # Salva le medie nel dizionario brand_year_averages
        brand_year_averages[(brand, year)] = feature_means

        # Opzionale: stampa le medie per il brand e anno corrente
        #print(f"Medie delle features per {brand} nel {year}:")
        #print(feature_means)

# Converti il dizionario brand_year_averages in un DataFrame per una visualizzazione più comoda
df_averages = pd.DataFrame.from_dict(brand_year_averages, orient='index')

# Mostra il DataFrame finale
#print(df_averages)

# Opzionale: salva il DataFrame su disco in formato CSV
#df_averages.to_csv('feature_averages_per_brand_year.csv')

In [ ]:
df_averages

In [ ]:
df_averages.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df_averages = df_averages.reset_index(drop=False)

# Lista dei brand disponibili nel DataFrame
brands = df_averages['level_0'].unique()  # level_0 contiene i brand

# Itera su ciascun brand
for brand in brands:
    # Filtra il DataFrame per il brand corrente
    df_brand = df_averages[df_averages['level_0'] == brand]
    
    # Filtra il DataFrame per includere solo le colonne battery e screen
    df_brand_filtered = df_brand[['level_1'] + ['battery', 'screen']]  # 'level_1' contiene l'anno

    # Rinominare le colonne se necessario
    df_brand_filtered.rename(columns={'level_1': 'year'}, inplace=True)

    # Converti la colonna 'year' in formato numerico
    df_brand_filtered['year'] = df_brand_filtered['year'].astype(int)

    # Raggruppa per anno per ottenere la media delle features per ogni anno (se ci fossero dati duplicati)
    df_grouped_brand = df_brand_filtered.groupby('year').mean()
    df_cumulative = df_grouped_brand.expanding().mean()

    # Crea il grafico di linee per ogni feature per il brand corrente
    plt.figure(figsize=(6,4))

    for feature in ['battery', 'screen']:
        plt.plot(df_cumulative.index, df_cumulative[feature], marker='o', label=feature)

    # Aggiungi titolo e etichette
    plt.title(f"trend screen e battery per {brand} (2013-2018)")
    plt.xlabel("Anno")
    plt.ylabel("Media cumulata dei Punteggi delle Features")
    plt.legend(title="Features", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)

    # Mostra il grafico
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / f'{brand}_features_trend.png', format='png')  # Salva con il nome del brand
    
    plt.show()

1. Le feature 'screen'e 'battery' per il brand BlackBerry hanno mostrato un graduale miglioramento nella percezione dei consumatori, come indicato dall'aumento costante del sentiment cumulativo dal 2013 al 2018. Questo riflette probabilmente i miglioramenti tecnologici e l'importanza crescente dello schermo e della batteria negli smartphone BlackBerry.

2. mentre per gli altri brand si nota un trend decrescente nella percezione dei consumatori per queste due features, il che può indicare il fatto che siano necessari dei miglioramenti.